In [ ]:
#meep version:1.30.0
import meep as mp
import numpy as np
import math
from matplotlib import pyplot as plt

%matplotlib inline

resolution = 100  

a = 0.46  
r = 0.105  
n = 2.805 
epsilon = n**2 

theta1 = 2.205   # These values can be adjusted. 
theta2 = -2.205  
theta1_rad = math.radians(theta1)
theta2_rad = math.radians(theta2)

sx = 16
sy = 15

nonpml_vol = mp.Volume(mp.Vector3(), size=mp.Vector3(sx-4,sy-4))

cell = mp.Vector3(sx, sy, 0) 

blk = mp.Block(size=mp.Vector3(mp.inf, mp.inf, mp.inf), material=mp.Medium(epsilon=epsilon))
geometry = [blk]

def generate_triangular_lattice_points(nx=21, ny=21):
    points = []
    
    n_rows = ny - 1
    n_cols = nx - 1
    even_flag = 0  
    
    for i in range(-n_rows//2, n_rows//2 + 1):
        if even_flag == 0:
            for j in range(-n_cols//2, n_cols//2 + 1):
                x = j * a
                y = i * a * math.sqrt(3)/2
                points.append(mp.Vector3(x, y))
        else:
            for j in range(-(n_cols-1)//2, (n_cols-1)//2 + 1):
                x = j * a + a/2
                y = i * a * math.sqrt(3)/2
                points.append(mp.Vector3(x, y))
        
        even_flag = 1 - even_flag
    
    return points

lattice_points = generate_triangular_lattice_points()

for p in lattice_points:
    x_rot = p.x * math.cos(theta1_rad) - p.y * math.sin(theta1_rad)
    y_rot = p.x * math.sin(theta1_rad) + p.y * math.cos(theta1_rad)
    geometry.append(mp.Cylinder(
        radius=r,
        center=mp.Vector3(x_rot, y_rot), 
        material=mp.Medium(epsilon=1),
        height=mp.inf
    ))

for p in lattice_points:
    x_rot = p.x * math.cos(theta2_rad) - p.y * math.sin(theta2_rad)
    y_rot = p.x * math.sin(theta2_rad) + p.y * math.cos(theta2_rad)
    geometry.append(mp.Cylinder(
        radius=r,
        center=mp.Vector3(x_rot, y_rot),
        material=mp.Medium(epsilon=1),
        height=mp.inf
    ))

def create_sector(center, radius, start_angle_deg, end_angle_deg, material, num_points=30):
    start_angle = math.radians(start_angle_deg)
    end_angle = math.radians(end_angle_deg)
    
    vertices = [center] 
    
    for i in range(num_points + 1):
        angle = start_angle + i * (end_angle - start_angle) / num_points
        point = center + mp.Vector3(radius * math.cos(angle), radius * math.sin(angle) )
        vertices.append(point)
    
    vertices.append(center)  
    
    sector = mp.Prism(vertices, height=mp.inf, material=mp.Medium(epsilon=epsilon))
    return sector

sector1 = create_sector(
    center=mp.Vector3(0, 0),
    radius=0.12,                           # This value must be a little larger than 0.105.
    start_angle_deg=60,
    end_angle_deg=120,
    material=mp.Medium(epsilon=epsilon)
)
geometry.append(sector1)

sector2 = create_sector(
    center=mp.Vector3(0, 0),
    radius=0.12,                           # This value must be a little larger than 0.105.
    start_angle_deg=240,
    end_angle_deg=300,
    material=mp.Medium(epsilon=epsilon)
)
geometry.append(sector2)

pml_layers = [mp.PML(2.0)]

fcen = 0.628  # pulse center frequency
df = 0.15  # pulse frequency width

np.random.seed(47)
sources = []
num_sources = 10
for _ in range(num_sources):
    sources.append(mp.Source(
        src=mp.GaussianSource(fcen, fwidth=df),
        component=mp.Hz,
        center=mp.Vector3(np.random.uniform(-0.3, 0.3), 
                         np.random.uniform(-0.3, 0.3))
    ))

sim = mp.Simulation(
    cell_size=cell,
    geometry=geometry,
    boundary_layers=pml_layers,
    sources=sources,
    resolution=resolution,
)

dft_obj = sim.add_dft_fields(
    [mp.Hz], 
    [0.62486, 0.62923],  # These values can be adjusted.  
    where=nonpml_vol,
    yee_grid=False
)

dft_obj2 = sim.add_dft_fields(
    [mp.Ex, mp.Ey, mp.Ez],  
    [0.62486, 0.62923], # These values can be adjusted. 
    where=nonpml_vol,
    yee_grid=False
)

f = plt.figure(figsize=(12, 10), dpi=1200)
sim.plot2D(ax=f.gca())
plt.tight_layout()
plt.show()

sim.run(
mp.after_sources(mp.Harminv(mp.Hz, mp.Vector3(0.1,0.1), fcen, df)),
until=750)

hz_data_f1 = sim.get_dft_array(dft_obj, mp.Hz, 0)  
hz_data_f2 = sim.get_dft_array(dft_obj, mp.Hz, 1)  

hz_real_f1 = np.real(hz_data_f1)
hz_real_f2 = np.real(hz_data_f2)

ex_data_f1 = sim.get_dft_array(dft_obj2, mp.Ex, 0)
ey_data_f1 = sim.get_dft_array(dft_obj2, mp.Ey, 0)
ez_data_f1 = sim.get_dft_array(dft_obj2, mp.Ez, 0)
e_magnitude_f1 = np.sqrt(
    np.abs(ex_data_f1)**2 + 
    np.abs(ey_data_f1)**2 + 
    np.abs(ez_data_f1)**2
)

ex_data_f2 = sim.get_dft_array(dft_obj2, mp.Ex, 1)
ey_data_f2 = sim.get_dft_array(dft_obj2, mp.Ey, 1)
ez_data_f2 = sim.get_dft_array(dft_obj2, mp.Ez, 1)
e_magnitude_f2 = np.sqrt(
    np.abs(ex_data_f2)**2 + 
    np.abs(ey_data_f2)**2 + 
    np.abs(ez_data_f2)**2
)

extent_x = (-(sx-4)/2, (sx-4)/2)
extent_y = (-(sy-4)/2, (sy-4)/2)
extent = [extent_x[0], extent_x[1], extent_y[0], extent_y[1]]

fig, axes = plt.subplots(2, 2, figsize=(16, 12), dpi=1200)

cmap = 'RdBu_r' 

ax1 = axes[0, 0]
im1 = ax1.imshow(hz_real_f1.transpose(), 
                 cmap=cmap, 
                 interpolation='spline36',
                 origin='lower',
                 extent=extent,
                 aspect='auto')
plt.colorbar(im1, ax=ax1, label='Hz Amplitude', shrink=0.8)

ax2 = axes[0, 1]
im2 = ax2.imshow(hz_real_f2.transpose(), 
                 cmap=cmap, 
                 interpolation='spline36',
                 origin='lower',
                 extent=extent,
                 aspect='auto')
plt.colorbar(im2, ax=ax2, label='Hz Amplitude', shrink=0.8)

ax3 = axes[1, 0]
im3 = ax3.imshow(e_magnitude_f1.transpose(), 
                 cmap='hot',  
                 interpolation='spline36',
                 origin='lower',
                 extent=extent,
                 aspect='auto')
plt.colorbar(im3, ax=ax3, label='|E| (a.u.)', shrink=0.8)

ax4 = axes[1, 1]
im4 = ax4.imshow(e_magnitude_f2.transpose(), 
                 cmap='hot',  
                 interpolation='spline36',
                 origin='lower',
                 extent=extent,
                 aspect='auto')
plt.colorbar(im4, ax=ax4, label='|E| (a.u.)', shrink=0.8)

plt.tight_layout()
plt.show()
